In [ ]:
!shred -u setup_colab.py
!shred -u setup_colab_general.py
!wget -q "https://github.com/jpcano1/python_utils/raw/main/setup_colab_general.py" -O setup_colab_general.py
!wget -q "https://github.com/jpcano1/python_utils/raw/main/ISIS_4825/setup_colab.py" -O setup_colab.py
import setup_colab as setup
setup.setup_workshop_12()

In [ ]:
# Basic Data Analysis Libraries
import numpy as np
import pandas as pd

# Basic OS Libraries
import copy
import os

# Basic Graphic Functions
import matplotlib.pyplot as plt
plt.style.use("seaborn-deep")
import seaborn as sns

# Util Functions
from utils import general as gen
from utils import torch_utils
from utils import visualization_utils as vis
from utils import train_utils

# Loaders
from tqdm.auto import tqdm

# Data Augmentation Libraries
from albumentations import (Equalize, RandomBrightness, RandomGamma, 
                            Compose, Resize)

# PyTorch Libraries
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import functional as F

# Torchvision Functions
from torchvision.transforms.functional import to_tensor

# Summary Functions
from torchsummary import summary

# Dataset Creation and Splitting Functions
from sklearn.model_selection import ShuffleSplit
from torch.utils.data import Dataset, DataLoader, Subset

# Computer Vision Libraries
import cv2

In [ ]:
train_dir = gen.create_and_verify(".", "data", "train_data")

In [ ]:
train_data_dir = gen.read_listdir(gen.read_listdir(train_dir)[0])
train_labels_dir = gen.read_listdir(gen.read_listdir(train_dir)[1])

In [ ]:
np.random.seed(1999)
random_sample = np.random.choice(range(len(train_data_dir)), 3)
random_sample

In [ ]:
plt.figure(figsize=(9, 9))

index = 1
for i in random_sample:
    path2img = train_data_dir[i]
    path2lab = train_labels_dir[i]
    
    X = np.load(path2img)
    y = np.load(path2lab)[..., 0]
    labeled_X = vis.get_labeled_image(X, y)

    plt.subplot(3, 3, index)
    gen.imshow(X, color=False, cmap="bone")

    plt.subplot(3, 3, index+1)
    gen.imshow(y, color=False)

    plt.subplot(3, 3, index+2)
    gen.imshow(labeled_X)

    index += 3

In [ ]:
plt.figure(figsize=(9, 9))

index = 1
for i in random_sample:
    path2img = train_data_dir[i]
    path2lab = train_labels_dir[i]
    
    X = np.load(path2img)
    y = np.load(path2lab)[..., 1]
    labeled_X = vis.get_labeled_image(X, y)

    plt.subplot(3, 3, index)
    gen.imshow(X, color=False, cmap="bone")

    plt.subplot(3, 3, index+1)
    gen.imshow(y, color=False)

    plt.subplot(3, 3, index+2)
    gen.imshow(labeled_X)

    index += 3

In [ ]:
transform_train = Compose([
    Resize(128, 128),
    Equalize(),
    RandomBrightness(),
    RandomGamma()
])

transform_val = Resize(128, 128)

In [ ]:
class KidneyDataset(Dataset):
    def __init__(self, path2data, transform=None, sample=2000, seed=None):
        self.train_data_dir = gen.read_listdir(gen.read_listdir(path2data)[0])
        self.train_labels_dir = gen.read_listdir(gen.read_listdir(path2data)[1])
        if seed:
            np.random.seed(seed)
        random_sample = np.random.choice(len(self.train_data_dir), sample)
        self.train_data_dir = self.train_data_dir[random_sample]
        self.train_labels_dir = self.train_labels_dir[random_sample]

        self.transform = transform

    def __len__(self):
        return len(self.train_data_dir)

    def __getitem__(self, index):
        path2img = self.train_data_dir[index]
        path2lab = self.train_labels_dir[index]

        X = np.load(path2img)
        y = np.load(path2lab)

        if self.transform:
            augmented = self.transform(image=X, mask=y)
            X = augmented["image"]
            y = augmented["mask"]
        X = to_tensor(X)
        y = 255. * to_tensor(y)
        return X, y

In [ ]:
kidney_ds1 = KidneyDataset(train_dir, seed=1234, transform=transform_train, 
                           sample=8200)
kidney_ds2 = KidneyDataset(train_dir, seed=1234, transform=transform_val, 
                           sample=8200)

In [ ]:
ss_data = ShuffleSplit(n_splits=2, test_size=0.2, random_state=1234)

In [ ]:
indices = range(len(kidney_ds1))

In [ ]:
for train_index, val_index in ss_data.split(indices):
    pass

In [ ]:
train_data = Subset(kidney_ds1, train_index)
val_data = Subset(kidney_ds2, val_index)

In [ ]:
train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
val_dl = DataLoader(val_data, batch_size=16)

# **Auntoencoder**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = torch_utils.Autoencoder(1, 2, 16, 4, bn=1, jump=2)
if os.path.exists("./models/autoencoder.pt"):
    model.load_state_dict(torch.load("./models/autoencoder.pt"))
    print("Pretrained Layers Loaded!")

model = model.to(device)

In [ ]:
summary(model, (1, 128, 128))

# **U-Net**

In [ ]:
model = torch_utils.UNet(1, 2, 16, 5, bn=1, jump=2)
if os.path.exists("./models/unet.pt"):
    model.load_state_dict(torch.load("./models/unet.pt"))
    print("Pretrained Layers Loaded!")

model = model.to(device)

In [ ]:
summary(model, (1, 128, 128))

## **Training**

In [ ]:
opt = optim.Adam(model.parameters(), lr=5e-3)
lr_scheduler = ReduceLROnPlateau(opt, mode="min", factor=0.5,
                                 patience=6, verbose=1)

weights_dir= "./models/"
if not os.path.exists(weights_dir):
        os.makedirs(weights_dir)

In [ ]:
args_train = (
    1, train_utils.loss_func, opt, train_dl, val_dl, False,
    lr_scheduler, weights_dir + "weights.pt", device
)

In [ ]:
out_model, loss_history, acc_history = train_utils.train(model, *args_train)

## **Predicción**

In [ ]:
test_dir = gen.create_and_verify(".", "data", "test_data")

test_data_dir = gen.read_listdir(gen.read_listdir(test_dir)[0])
test_labels_dir = gen.read_listdir(gen.read_listdir(test_dir)[1])

In [ ]:
weights_dir = gen.create_and_verify(".", "models", "unet.pt")

In [ ]:
model.load_state_dict(torch.load(weights_dir))
model = model.eval().to(device)

In [ ]:
np.random.seed(420)
random_sample = np.random.choice(len(test_data_dir), 3)
random_sample

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(len(random_sample)):
    rnd_idx = random_sample[i]
    X = np.load(test_data_dir[rnd_idx])
    y_true = np.load(test_labels_dir[rnd_idx])[..., 0]
    X = cv2.resize(X, (128, 128), cv2.INTER_NEAREST)
    X_t = to_tensor(X).unsqueeze(0).to(device)
    y_pred = model(X_t)
    y_pred = y_pred.squeeze(0)
    y_pred = y_pred[0].cpu().detach().numpy() > .5
    
    plt.subplot(3, 4, 1 + i*4)
    gen.imshow(X, color=False, cmap="bone", title="Image")

    plt.subplot(3, 4, 2 + i*4)
    gen.imshow(y_pred, color=False, title="Predicted Kidney")

    plt.subplot(3, 4, 3 + i*4)
    gen.imshow(vis.mark_boundaries(X, y_pred), title="Boundary")

    plt.subplot(3, 4, 4 + i*4)
    gen.imshow(y_true, color=False, title="True Label")

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(len(random_sample)):
    rnd_idx = random_sample[i]
    X = np.load(test_data_dir[rnd_idx])
    y_true = np.load(test_labels_dir[rnd_idx])[..., 1]
    X = cv2.resize(X, (128, 128), cv2.INTER_NEAREST)
    X_t = to_tensor(X).unsqueeze(0).to(device)
    y_pred = model(X_t)
    y_pred = y_pred.squeeze(0)
    y_pred = y_pred[1].cpu().detach().numpy() > .5
    
    plt.subplot(3, 4, 1 + i*4)
    gen.imshow(X, color=False, cmap="bone", title="Image")

    plt.subplot(3, 4, 2 + i*4)
    gen.imshow(y_pred, color=False, title="Predicted Tumor")

    plt.subplot(3, 4, 3 + i*4)
    gen.imshow(vis.mark_boundaries(X, y_pred), title="Boundary")

    plt.subplot(3, 4, 4 + i*4)
    gen.imshow(y_true, color=False, title="True Label")